In [1]:
import pandas as pd
import numpy as np
import os
import sys
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import json

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
os.chdir('../')
os.getcwd()

# root_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(root_dir)

from utils.utils     import *
from utils.constants import *
from utils.metrics   import *
from src.model_tuning import *

In [3]:
path_ = os.getcwd()

In [4]:
os.getcwd()

'F:\\2022_Paper_03232023\\10 Room Occupancy Estimation\\Code'

In [5]:

y_train = pd.read_csv(path_ + '\\data' + '\\y_train.csv')

y_test = pd.read_csv(path_ + '\\data' + '\\y_test.csv')


stack_train = pd.read_csv(path_ + '\\data' + '\\X_train.csv')
stack_test  = pd.read_csv(path_ + '\\data' + '\\X_test.csv')


# y_train = pd.read_csv(get_absolute_path('y_train.csv', 'data'))
# y_test = pd.read_csv(get_absolute_path('y_test.csv', 'data'))


# stack_train = pd.read_csv(get_absolute_path('stacked_X_tr.csv', 'data'))
# stack_test  = pd.read_csv(get_absolute_path('stacked_X_te.csv', 'data'))




In [6]:
stack_train

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
0,25.19,25.19,24.63,25.56,0,0,0,0,0.08,0.05,0.06,0.05,360,0.000000,0,0
1,25.38,25.38,24.81,25.75,0,0,0,0,0.07,0.05,0.06,0.06,365,0.000000,0,0
2,25.56,25.63,25.31,25.69,0,0,0,0,0.07,0.05,0.05,0.08,530,-2.592308,0,0
3,25.50,25.50,25.06,25.88,0,0,0,0,0.07,0.04,0.07,0.06,390,-0.642308,0,0
4,25.38,25.38,24.88,25.75,0,0,0,0,0.07,0.05,0.06,0.10,355,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8098,25.31,25.31,25.13,25.88,17,20,75,54,0.07,0.05,0.06,0.09,355,0.000000,0,0
8099,25.13,25.13,24.56,25.44,5,5,28,19,0.07,0.04,0.06,0.06,360,0.000000,0,0
8100,25.06,25.06,24.63,25.50,9,10,45,30,0.07,0.04,0.06,0.09,355,0.000000,0,0
8101,26.31,26.69,26.06,26.44,150,236,180,10,1.03,0.27,0.23,0.11,1160,3.165385,0,1


In [7]:

# stack_train = stack_train.astype(column_data_extended_types)
# stack_test = stack_test.astype(column_data_extended_types)

In [8]:
######## Feature Engineering ##########

# Select numeric and categorical columns
numeric_columns = stack_train.select_dtypes(include=['float64', 'int64']).columns

categorical_columns = [#'Date', 
                       # 'Location_ID',
                    #    'Year',
                       # 'Month',
                       # 'Week',
                       # 'Weekday',
                       # 'Season'
                       ]  # Add any categorical columns here


# Create preprocessing transformers
numeric_transformer = StandardScaler(with_mean = False, with_std = False)  # we can use other scalers as well
categorical_transformer = OneHotEncoder(drop=None)  # Use one-hot encoding for categorical columns



# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Fit the preprocessor on training data and transform both train and test data
X_train_preprocessed = preprocessor.fit_transform(stack_train)
X_test_preprocessed  = preprocessor.transform(stack_test)

In [9]:
X_train_preprocessed

array([[-0.74401722, -0.60102236, -0.98929699, ...,  0.01289966,
        -0.31109253, -0.29159811],
       [-0.20331256, -0.28002188, -0.56870866, ...,  0.01289966,
        -0.31109253, -0.29159811],
       [ 0.30893395,  0.14234718,  0.59959229, ..., -2.20336051,
        -0.31109253, -0.29159811],
       ...,
       [-1.11397304, -0.82065427, -0.98929699, ...,  0.01289966,
        -0.31109253, -0.29159811],
       [ 2.44329444,  1.933192  ,  2.3520437 , ...,  2.71910458,
        -0.31109253,  3.42937748],
       [-0.40251954, -0.39828521, -0.40514652, ...,  0.01289966,
        -0.31109253, -0.29159811]])

In [10]:

# # Get the column names after one-hot encoding
# categorical_encoded_columns = preprocessor.named_transformers_['cat']\
#                                     .get_feature_names_out(input_features=categorical_columns)

In [11]:



# # Convert X_train_preprocessed and X_test_preprocessed to DataFrames

# X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed.toarray(), columns=np.concatenate([numeric_columns, categorical_encoded_columns]))
# X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed.toarray(), columns=np.concatenate([numeric_columns, categorical_encoded_columns]))

X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=np.concatenate([numeric_columns]))
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=np.concatenate([numeric_columns]))



# X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed.toarray(), columns=np.concatenate([numeric_columns]))
# X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed.toarray(), columns=np.concatenate([numeric_columns]))


# X_train_preprocessed_df = X_train_preprocessed_df[column_names_raw]
# X_test_preprocessed_df = X_test_preprocessed_df[column_names_raw]



In [12]:
# scoring=make_scorer(lambda y_true, y_pred: -mean_squared_error(y_true, y_pred, squared=False))


scoring = 'roc_auc_ovr_weighted'

# Models

In [13]:
######### XGBoost #########

# best_xgb_file = get_absolute_path(
#     file_name = 'best_xgb_model.joblib'
#     , rel_path = 'results'
# )

best_xgb_file =  path_ + '\\results' + '\\best_xgb_model.joblib'


xgb_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid={
            'max_depth': [3, 5, 7],
            'learning_rate': [0.1, 0.01],
            'n_estimators': [100, 200, 500, 800],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'gamma': [0, 0.1, 0.5],
            'min_child_weight': [1, 5, 10, 20],
            'random_state': [RANDOM_SEED]
        },
        model=xgb.XGBClassifier(objective='multi:softmax', random_state=RANDOM_SEED),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_xgb_file,
        cv=5
) 


print("Success of xgboost!")


Fitting 5 folds for each of 1152 candidates, totalling 5760 fits
best_params: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 800, 'random_state': 42, 'subsample': 1.0}
eval_metrics: {'f1_weighted': 0.9970266322461073, 'auc_weighted': 0.9999398694500884, 'accuracy_score': 0.9970384995064165, 'balanced_accuracy_score': 0.989808362369338, 'precision_score': 0.9970336550774833, 'recall_score': 0.9970384995064165, 'jaccard_score': 0.9941242530904206}
Success of xgboost!


In [14]:
# xgb_result

In [15]:
path_ + '\\results' + "\\xgb_result.json"

'F:\\2022_Paper_03232023\\10 Room Occupancy Estimation\\Code\\results\\xgb_result.json'

In [16]:
# import json
  
# with open('xgb_result.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(str(xgb_result)))

In [17]:
# 'xgb_result.txt'

In [18]:
# with open((path_ + '\\results' + "\\xgb_result.json"), "w") as outfile:
#     json.dump(xgb_result, outfile)


In [19]:

######### Random Forest #########

# best_rf_file = get_absolute_path(
#     file_name = 'best_rf_model.joblib'
#     , rel_path = 'results'
# )

best_rf_file =  path_ + '\\results' + '\\best_rf_model.joblib'


rf_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid={
           'n_estimators': [50, 100, 200, 500],
           'max_depth': [None, 3, 5, 10, 20],
           'min_samples_split': [1, 2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'random_state': [RANDOM_SEED]
        },
        model=RandomForestClassifier(random_state=RANDOM_SEED),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_rf_file,
        cv=5
) 

print("Success of rf!")



Fitting 5 folds for each of 240 candidates, totalling 1200 fits


D:\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
300 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "D:\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "D:\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "D:\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validat

best_params: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'random_state': 42}


D:\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


eval_metrics: {'f1_weighted': 0.9985166722727276, 'auc_weighted': 0.9999593996313667, 'accuracy_score': 0.9985192497532083, 'balanced_accuracy_score': 0.9949041811846691, 'precision_score': 0.9985160034745045, 'recall_score': 0.9985192497532083, 'jaccard_score': 0.9970552887916172}
Success of rf!


In [20]:
# with open((path_ + '\\results' + "\\rf_result.json"), "w") as outfile:
#     json.dump(str(rf_result), outfile)

In [21]:


######### MLP #########

# best_mlp_file = get_absolute_path(
#     file_name = 'best_mlp_model.joblib'
#     , rel_path = 'results'
# )

best_mlp_file =  path_ + '\\results' + '\\best_mlp_model.joblib'



mlp_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid= {
            'hidden_layer_sizes': list(zip([1000,2000, 3000, 5000], [300, 500, 800, 1000])),
            #'max_iter': [50, 100],
            'activation': ['tanh', 'tanh'], #'activation': ['tanh', 'relu'],
            #'solver': ['sgd', 'adam'],
            #'alpha': [0.0001, 0.05],
            'learning_rate': ['adaptive'],
            'random_state': [RANDOM_SEED]
        },
        model=MLPClassifier(random_state=RANDOM_SEED),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_mlp_file,
        cv=5
) 

print("Success of MLP!")



Fitting 5 folds for each of 8 candidates, totalling 40 fits


D:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'activation': 'tanh', 'hidden_layer_sizes': (2000, 500), 'learning_rate': 'adaptive', 'random_state': 42}


D:\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


eval_metrics: {'f1_weighted': 0.9970413059420704, 'auc_weighted': 0.9998849686481783, 'accuracy_score': 0.9970384995064165, 'balanced_accuracy_score': 0.991962308432314, 'precision_score': 0.9970754678816185, 'recall_score': 0.9970384995064165, 'jaccard_score': 0.9941652553325802}
Success of MLP!


In [22]:
# with open((path_ + '\\results' + "\\mlp_result.json"), "w") as outfile:
#     json.dump(str(mlp_result), outfile)

In [23]:

######### ElasticNet #########

# best_lin_file = get_absolute_path(
#     file_name = 'best_lin_model.joblib'
#     , rel_path = 'results'
# )

best_lin_file =  path_ + '\\results' + '\\best_lin_model.joblib'


lin_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid={
            'loss': ['log_loss'],
            'penalty': ['l2', 'l1', 'elasticnet'],
            'random_state': [RANDOM_SEED]
        },
        model=SGDClassifier(random_state=RANDOM_SEED).fit(X_train_preprocessed_df,y_train),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_lin_file,
        cv=5
) 

print("Success of ElasticNet!")


Fitting 5 folds for each of 3 candidates, totalling 15 fits


D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'loss': 'log_loss', 'penalty': 'elasticnet', 'random_state': 42}
eval_metrics: {'f1_weighted': 0.981767346555007, 'auc_weighted': 0.9965431914818418, 'accuracy_score': 0.9817374136229022, 'balanced_accuracy_score': 0.941546419210126, 'precision_score': 0.981857068711013, 'recall_score': 0.9817374136229022, 'jaccard_score': 0.9662288136313761}
Success of ElasticNet!


In [24]:
# with open((path_ + '\\results' + "\\lin_result.json"), "w") as outfile:
#     json.dump(str(lin_result), outfile)

In [25]:

######### lightGBM #########

## rename the features for an feature name error (only for lightGBM).
# import re
# X_train_preprocessed_df = X_train_preprocessed_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
# X_test_preprocessed_df = X_test_preprocessed_df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


# best_best_lightGBM_file = get_absolute_path(
#     file_name = 'best_lightGBM_model.joblib'
#     , rel_path = 'results'
# )

best_lightGBM_file =  path_ + '\\results' + '\\best_lightGBM_model.joblib'


lightGBM_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid={
                'max_depth': [3, 5, 7, 15],
                'learning_rate': [0.5, 0.1, 0.01, 0.001],
                'num_leaves': [20, 31, 50, 100,200],
                'n_estimators': [50, 100, 200, 500, 800],
                'random_state': [RANDOM_SEED]
        },
        model=lgb.LGBMClassifier(objective='multiclass', random_state=RANDOM_SEED).fit(X_train_preprocessed_df,y_train),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_lightGBM_file,
        cv=5
) 

print("Success of LightGBM!")


D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Fitting 5 folds for each of 400 candidates, totalling 2000 fits


D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


best_params: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 50, 'num_leaves': 31, 'random_state': 42}


D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


eval_metrics: {'f1_weighted': 0.9970295173893192, 'auc_weighted': 0.999939713061599, 'accuracy_score': 0.9970384995064165, 'balanced_accuracy_score': 0.9900696864111498, 'precision_score': 0.9970312679313816, 'recall_score': 0.9970384995064165, 'jaccard_score': 0.9941296880338865}
Success of LightGBM!


In [26]:
# with open((path_ + '\\results' + "\\lightGBM_model.json"), "w") as outfile:
#     json.dump(str(lightGBM_result), outfile)

In [27]:
######### LinearDiscriminantAnalysis #########


best_LinearDiscriminantAnalysis_file =  path_ + '\\results' + '\\best_LinearDiscriminantAnalysis_model.joblib'


LDA_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid={
            'solver': ['svd', 'lsqr', 'eigen']

        },
        model=LinearDiscriminantAnalysis(),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_LinearDiscriminantAnalysis_file,
        cv=5
) 

print("Success of LinearDiscriminantAnalysis!")



Fitting 5 folds for each of 3 candidates, totalling 15 fits
best_params: {'solver': 'lsqr'}
eval_metrics: {'f1_weighted': 0.9815939195291642, 'auc_weighted': 0.997138502876409, 'accuracy_score': 0.9817374136229022, 'balanced_accuracy_score': 0.9421563241817011, 'precision_score': 0.9820144958356644, 'recall_score': 0.9817374136229022, 'jaccard_score': 0.9658375549147866}
Success of LinearDiscriminantAnalysis!


D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [28]:
######### SVC #########


best_SVC_file =  path_ + '\\results' + '\\best_SVC_model.joblib'


LDA_result = hyperparameter_tuning(
        X_train=X_train_preprocessed_df,
        y_train=y_train,
        X_test=X_test_preprocessed_df,
        y_test=y_test,
        param_grid={
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
           # 'n_estimators': [50, 100, 200, 500],
           # 'max_depth': [None, 3, 5, 10, 20],
           # 'min_samples_split': [1, 2, 5, 10],
            # 'min_samples_leaf': [1, 2, 4],
            # 'random_state': [RANDOM_SEED]
        },
        model=SVC(probability = True),
        scoring=scoring,
        eval_func=compute_metrics,
        file_path=best_SVC_file,
        cv=5
) 

print("Success of SVC!")



Fitting 5 folds for each of 4 candidates, totalling 20 fits


D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


best_params: {'kernel': 'rbf'}


D:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


eval_metrics: {'f1_weighted': 0.9921396688109194, 'auc_weighted': 0.999511643538035, 'accuracy_score': 0.9921026653504442, 'balanced_accuracy_score': 0.9763461452150775, 'precision_score': 0.9921992974073819, 'recall_score': 0.9921026653504442, 'jaccard_score': 0.9848225340388622}
Success of SVC!
